In [ ]:
%load_ext kamu

In [ ]:
%import_dataset ca.vancouver.opendata.property.block-outlines --alias blocks

In [ ]:
%import_dataset ca.vancouver.opendata.property.local-area-boundaries --alias hoods

In [ ]:
blocks.printSchema()
hoods.printSchema()

In [ ]:
%%sql
CREATE OR REPLACE TEMP VIEW blocks_by_hood AS (
    SELECT h.name, b.geometry
    FROM blocks as b
    INNER JOIN hoods as h
    ON ST_Intersects(b.geometry, h.geometry)
)

In [ ]:
%%sql -o df -n 1000000
SELECT 
    ST_AsText(geometry) as geometry, name, RAND() as median_value
FROM blocks_by_hood

In [ ]:
%%local
import shapely.wkt
import geojson

def df_to_geojson(df, geom='geometry', props=None):
    if props is None:
        props = [
            c for c in df.columns
            if c != geom
        ]

    return geojson.FeatureCollection([
        geojson.Feature(
            geometry=shapely.wkt.loads(r[geom]),
            properties={p: r[p] for p in props}
        )
        for _, r in df.iterrows()
    ])

In [ ]:
%%local
gj = df_to_geojson(df)

In [ ]:
%%local
import os
from mapboxgl.viz import *
from mapboxgl.utils import *

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

In [ ]:
%%local
viz = ChoroplethViz(
    gj,
    style='mapbox://styles/mapbox/dark-v10',
    center=(-123.1207, 49.2827),
    zoom=10,
    access_token=token,
    color_property='median_value',
    color_stops=create_color_stops([0.25, 0.5, 0.75, 1.0], colors='YlOrRd'),
    line_stroke='solid',
    line_width=0.1,
    line_color='rgb(128,0,38)',
    opacity=0.8,
    legend_layout='horizontal',
    legend_key_shape='bar',
    legend_key_borders_on=False)

viz.show()

In [ ]:
%%local
# adjust view angle
viz.bearing = -15
viz.pitch = 45

# add extrusion to viz using interpolation keyed on density in GeoJSON features
viz.height_property = 'median_value'
viz.height_stops = create_numeric_stops([0, 1], 0, 500)
viz.height_function_type = 'interpolate'
viz.opacity = 1

# render again
viz.show()